# ETL DataSUS APAC Drugs

ELT com PySUS

Author: Rodrigo Barreiro

# Testing if PySUS is working

In [1]:
import pandas as pd
from collections import Counter
from pathlib import Path
import re
from pysus import SIA

In [ ]:
sia = SIA().load() # Loads the files from DATASUS

In [ ]:
sia.metadata

In [ ]:
sia.groups

In [ ]:
print(f'All files for APAC Medicamentos: {len(sia.get_files(["AM"]))}')

## Calculating Files and Size

In [ ]:
all_apac_quimio_sia = len(sia.get_files("AM", year=2022))
print(f'Number of files for AQ in 2022: {all_apac_quimio_sia}')

In [ ]:
aq_files_2022 = sia.get_files("AM", year=2022)
Counter([ str(x)[2:4] for x in aq_files_2022 ])

In [ ]:
# Countig file size
files = sia.get_files("AM", year=2022)
sizes = [ sia.describe(my_file)['size'] for my_file in files ]


# Example list
sizes = ['37.9 kB', '1.2 MB', '550 B', '2.1 MB', '900 kB']

# Conversion to bytes
unit_multipliers = {
    'B': 1,
    'kB': 1024,
    'MB': 1024 ** 2,
    'GB': 1024 ** 3
}

total_bytes = 0

for s in sizes:
    match = re.match(r'([\d.]+)\s*(B|kB|MB|GB)', s)
    if match:
        number, unit = match.groups()
        bytes_size = float(number) * unit_multipliers[unit]
        total_bytes += bytes_size
    else:
        print(f"Unrecognized size format: {s}")

# Optional: convert back to human-readable
def human_readable(size_bytes):
    for unit in ['B', 'kB', 'MB', 'GB']:
        if size_bytes < 1024 or unit == 'GB':
            return f"{size_bytes:.2f} {unit}"
        size_bytes /= 1024

print(f"Total: {human_readable(total_bytes)}")
print("NOTE: This is the size of dbc file (compressed).")

## Extract
### Downlaod all APAC drugs from 2022 of all BR States

In [ ]:
# Download all files of 2022
# sia.download(files, local_dir='../data/raw/apac-am')

Check if all files were downloaded

In [ ]:
download_dir = Path('../data/raw/apac-am')
all_downloaded_files = [f.name for f in download_dir.iterdir() if f.suffix == '.parquet']
print(f'All downloaded files: {len(all_downloaded_files)} of {all_apac_quimio_sia} of SIA')

## Transform
### Convert to dataframe and merge all files into one

In [ ]:
all_data_df_list = [ x.to_dataframe() for x in sia.download(files, local_dir='../data/raw/apac-am') ] 

In [ ]:
print("Check type:")
print([type(x) for x in all_data_df_list[:3]])
print('\n')
print("Check dataframe n:")
print(len(all_data_df_list))

In [ ]:
# Concatenate to a single dataframe
combined_df = pd.concat(all_data_df_list, ignore_index=True)
del all_data_df_list

print(f"Rows (Atendimentos) {combined_df.shape[0]:,.0f} \nColumns (Campos) {combined_df.shape[1]:,.0f} ")

In [ ]:
combined_df.to_csv('../data/interim/apac-am-2022-full.csv', index = False)

In [ ]:
print(combined_df.columns.tolist())

In [ ]:
combined_df.head(10)

### Better naming


In [ ]:
new_names_df = pd.read_csv('../data/external/better_names_apac_am.csv', sep = ';')

my_dict = new_names_df.set_index('original_name')['new_name'].to_dict()
my_dict

In [ ]:
combined_df = combined_df.rename(columns=my_dict)
combined_df

In [ ]:
selected_fields = ["numero_apac","procedimento_principal","valor_total_apac","codigo_uf_municipio","cns_paciente","idade","sexo","raca_cor","motivo_saida_permanencia","data_ocorrencia","cid_principal","peso","altura","indicador_transplante","quantidade_transplantes","indicador_gestante"]

combined_df = combined_df[selected_fields].copy()


In [ ]:
combined_df.to_csv('../data/interim/apac-am-2022-short.csv', index = False)

### Add CID

In [29]:
# combined_df = pd.read_csv('../data/interim/apac-am-2022-short.csv').sample(frac = 0.1, axis = 0)
combined_df = pd.read_csv('../data/interim/apac-am-2022-short.csv')

In [30]:
cid10_data = pd.read_csv('../data/external/CID-10-CATEGORIAS.CSV.utf8',sep = ';')
cid10_data = cid10_data[['CAT','DESCRICAO']]

cid10_data.rename(columns={"CAT": 'CID10', "DESCRICAO":"cid10_main_descricao"}, inplace=True)
cid10_data

,CID10,cid10_main_descricao
0,A00,Cólera
1,A01,Febres tifóide e paratifóide
2,A02,Outras infecções por Salmonella
3,A03,Shiguelose
4,A04,Outras infecções intestinais bacterianas
...,...,...
2040,U80,Agente resistente à penicilina e antibióticos ...
2041,U81,Agente resistente à vancomicina e antibióticos...
2042,U88,Agente resistente a múltiplos antibióticos
2043,U89,Agente resistente a outros antibióticos e a an...


In [31]:
combined_df['cid_principal_main_categ'] = combined_df['cid_principal'].str[:3]
combined_df['cid_principal_main_categ']

0           F20
1           F20
2           E23
3           E22
4           E22
           ... 
27162599    E22
27162600    N18
27162601    E23
27162602    F20
27162603    G20
Name: cid_principal_main_categ, Length: 27162604, dtype: object

In [32]:
combined_df = pd.merge(combined_df, cid10_data, how = 'left', left_on = 'cid_principal_main_categ', right_on = 'CID10')

In [33]:
# Output most common CID table to GPT create simple names
summary_cid = combined_df.groupby('cid10_main_descricao').size().reset_index(name='n').sort_values('n',  ascending=False).head(30)
# summary_cid.head(50).to_csv('../data/interim/top50cids.csv', index = False)   
combined_df

,numero_apac,procedimento_principal,valor_total_apac,codigo_uf_municipio,cns_paciente,idade,sexo,raca_cor,motivo_saida_permanencia,data_ocorrencia,cid_principal,peso,altura,indicador_transplante,quantidade_transplantes,indicador_gestante,cid_principal_main_categ,CID10,cid10_main_descricao
0,1221200228260,604230028.0,0.00,120020,{{{}||,19,M,3,21,NaN,F208,83,177,N,0,N,F20,F20,Esquizofrenia
1,1221200222231,604230028.0,0.00,120040,{{~~|,14,M,3,21,NaN,F200,713,160,N,0,N,F20,F20,Esquizofrenia
2,1221200245089,604100019.0,0.00,120040,{}{{~{,48,F,3,21,NaN,E232,120,158,N,0,N,E23,E23,Hipofunção e outros transtornos da hipófise
3,1221200245232,604110030.0,256.58,120040,{{~~~||,10,F,3,21,NaN,E228,37,144,N,0,N,E22,E22,Hiperfunção da hipófise
4,1221200251854,604110030.0,256.58,120040,{{~{{|},8,F,3,21,NaN,E228,29,117,N,0,N,E22,E22,Hiperfunção da hipófise
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27162599,1722200329620,604030037.0,0.00,170210,{~{|~{{,37,M,1,21,NaN,E221,800,180,N,0,N,E22,E22,Hiperfunção da hipófise
27162600,1722200326715,604260016.0,0.00,172100,{{|{|{}{,42,M,1,21,NaN,N180,68,171,N,0,N,N18,N18,Insuficiência renal crônica
27162601,1722200314769,604610017.0,0.00,172100,{{}|}||,11,M,1,21,NaN,E230,28,134,N,0,N,E23,E23,Hipofunção e outros transtornos da hipófise
27162602,1722200313746,604230010.0,0.00,172100,{{{{},52,F,3,21,NaN,F200,58,160,N,0,N,F20,F20,Esquizofrenia


### Add Procedimentos (SIGTAP)

In [34]:
procedimentos = pd.read_csv('../data/external/procedimentos-sigtap.csv', sep = ';')
procedimentos['procedimento_principal'] = procedimentos['procedimento_principal'].astype('str')

procedimentos

,procedimento_principal,procedimento_principal_descricao
0,101010010,ATIVIDADE EDUCATIVA / ORIENTACAO EM GRUPO NA A...
1,101010028,ATIVIDADE EDUCATIVA / ORIENTACAO EM GRUPO NA A...
2,101010036,PRATICA CORPORAL / ATIVIDADE FISICA EM GRUPO
3,101010044,PRATICAS CORPORAIS EM MEDICINA TRADICIONAL CHI...
4,101010052,TERAPIA COMUNITARIA
...,...,...
5555,905010035,OCI AVALIACAO INICIAL EM OFTALMOLOGIA - A PART...
5556,905010043,OCI AVALIACAO DE RETINOPATIA DIABETICA
5557,905010051,OCI AVALIACAO INICIAL PARA ONCOLOGIA OFTALMOLO...
5558,905010060,OCI AVALIACAO DIAGNOSTICA EM NEURO OFTALMOLOGIA


In [35]:
combined_df['procedimento_principal'] = combined_df['procedimento_principal'].astype(str).str.split('.').str[0]
combined_df = pd.merge(combined_df, procedimentos, how = 'left', on = 'procedimento_principal')
combined_df.head(10)

,numero_apac,procedimento_principal,valor_total_apac,codigo_uf_municipio,cns_paciente,idade,sexo,raca_cor,motivo_saida_permanencia,data_ocorrencia,cid_principal,peso,altura,indicador_transplante,quantidade_transplantes,indicador_gestante,cid_principal_main_categ,CID10,cid10_main_descricao,procedimento_principal_descricao
0,1221200228260,604230028,0.00,120020,{{{}||,19,M,3,21,NaN,F208,83,177,N,0,N,F20,F20,Esquizofrenia,OLANZAPINA 10 MG (POR COMPRIMIDO)
1,1221200222231,604230028,0.00,120040,{{~~|,14,M,3,21,NaN,F200,713,160,N,0,N,F20,F20,Esquizofrenia,OLANZAPINA 10 MG (POR COMPRIMIDO)
2,1221200245089,604100019,0.00,120040,{}{{~{,48,F,3,21,NaN,E232,120,158,N,0,N,E23,E23,Hipofunção e outros transtornos da hipófise,"DESMOPRESSINA 0,1 MG/ML APLICACAO NASAL (POR F..."
3,1221200245232,604110030,256.58,120040,{{~~~||,10,F,3,21,NaN,E228,37,144,N,0,N,E22,E22,Hiperfunção da hipófise,"LEUPRORRELINA 3,75 MG INJETAVEL (POR FRASCO-AM..."
4,1221200251854,604110030,256.58,120040,{{~{{|},8,F,3,21,NaN,E228,29,117,N,0,N,E22,E22,Hiperfunção da hipófise,"LEUPRORRELINA 3,75 MG INJETAVEL (POR FRASCO-AM..."
5,1221200223870,604110030,256.58,120040,{~}{|~,8,F,3,21,NaN,E228,28,123,N,0,N,E22,E22,Hiperfunção da hipófise,"LEUPRORRELINA 3,75 MG INJETAVEL (POR FRASCO-AM..."
6,1222200004014,604400012,10.08,120020,{}{|{|{,78,M,4,21,NaN,N180,60,160,N,0,N,N18,N18,Insuficiência renal crônica,SEVELAMER 800 MG (POR COMPRIMIDO)
7,1221200250358,604470045,0.00,120040,{{}{|,65,F,3,21,NaN,N180,55,0,N,0,N,N18,N18,Insuficiência renal crônica,ALFAEPOETINA 4.000 UI INJETAVEL (POR FRASCO-AM...
8,1221200266506,604540019,0.00,120040,{|{||~{,70,M,99,21,NaN,G122,85,175,N,0,N,G12,G12,Atrofia muscular espinal e síndromes correlatas,RILUZOL 50 MG (POR COMPRIMIDO)
9,1222200001462,604510020,9.90,120040,{{~{|,11,M,3,21,NaN,F840,44,150,N,0,N,F84,F84,Transtornos globais do desenvolvimento,RISPERIDONA 2 MG (POR COMPRIMIDO)


### Add Estados

In [36]:
codigo_estados = pd.read_csv("../data/external/estados.csv")
codigo_estados.rename({'COD':'codigo_estado', 'NOME':'nome_estado', 'SIGLA':'sigla_estado'}, axis = 1, inplace=True)

codigo_estados


,codigo_estado,nome_estado,sigla_estado
0,35,São Paulo,SP
1,41,Paraná,PR
2,42,Santa Catarina,SC
3,43,Rio Grande do Sul,RS
4,50,Mato Grosso do Sul,MS
5,11,Rondônia,RO
6,12,Acre,AC
7,13,Amazonas,AM
8,14,Roraima,RR
9,15,Pará,PA


In [38]:
combined_df['codigo_estado'] = combined_df.astype(str).codigo_uf_municipio.str[:2].astype(int)

In [39]:
combined_df = pd.merge(combined_df, codigo_estados, how = 'left', on = 'codigo_estado')
combined_df

,numero_apac,procedimento_principal,valor_total_apac,codigo_uf_municipio,cns_paciente,idade,sexo,raca_cor,motivo_saida_permanencia,data_ocorrencia,...,indicador_transplante,quantidade_transplantes,indicador_gestante,cid_principal_main_categ,CID10,cid10_main_descricao,procedimento_principal_descricao,codigo_estado,nome_estado,sigla_estado
0,1221200228260,604230028,0.00,120020,{{{}||,19,M,3,21,NaN,...,N,0,N,F20,F20,Esquizofrenia,OLANZAPINA 10 MG (POR COMPRIMIDO),12,Acre,AC
1,1221200222231,604230028,0.00,120040,{{~~|,14,M,3,21,NaN,...,N,0,N,F20,F20,Esquizofrenia,OLANZAPINA 10 MG (POR COMPRIMIDO),12,Acre,AC
2,1221200245089,604100019,0.00,120040,{}{{~{,48,F,3,21,NaN,...,N,0,N,E23,E23,Hipofunção e outros transtornos da hipófise,"DESMOPRESSINA 0,1 MG/ML APLICACAO NASAL (POR F...",12,Acre,AC
3,1221200245232,604110030,256.58,120040,{{~~~||,10,F,3,21,NaN,...,N,0,N,E22,E22,Hiperfunção da hipófise,"LEUPRORRELINA 3,75 MG INJETAVEL (POR FRASCO-AM...",12,Acre,AC
4,1221200251854,604110030,256.58,120040,{{~{{|},8,F,3,21,NaN,...,N,0,N,E22,E22,Hiperfunção da hipófise,"LEUPRORRELINA 3,75 MG INJETAVEL (POR FRASCO-AM...",12,Acre,AC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27162599,1722200329620,604030037,0.00,170210,{~{|~{{,37,M,1,21,NaN,...,N,0,N,E22,E22,Hiperfunção da hipófise,"CABERGOLINA 0,5 MG (POR COMPRIMIDO)",17,Tocantins,TO
27162600,1722200326715,604260016,0.00,172100,{{|{|{}{,42,M,1,21,NaN,...,N,0,N,N18,N18,Insuficiência renal crônica,SACARATO DE HIDROXIDO FERRICO 100 MG INJETAVEL...,17,Tocantins,TO
27162601,1722200314769,604610017,0.00,172100,{{}|}||,11,M,1,21,NaN,...,N,0,N,E23,E23,Hipofunção e outros transtornos da hipófise,SOMATROPINA 4 UI INJETAVEL (POR FRASCO-AMPOLA),17,Tocantins,TO
27162602,1722200313746,604230010,0.00,172100,{{{{},52,F,3,21,NaN,...,N,0,N,F20,F20,Esquizofrenia,OLANZAPINA 5 MG (POR COMPRIMIDO),17,Tocantins,TO


### Add motivo saída

In [41]:
motivo_saida = pd.read_csv('../data/external/motivo-saida.csv')
motivo_saida['motivo_saida'] = motivo_saida['motivo_saida'].astype(int)
combined_df = pd.merge(combined_df, motivo_saida, how = 'left', left_on = 'motivo_saida_permanencia', right_on = 'motivo_saida')
combined_df

,numero_apac,procedimento_principal,valor_total_apac,codigo_uf_municipio,cns_paciente,idade,sexo,raca_cor,motivo_saida_permanencia,data_ocorrencia,...,indicador_gestante,cid_principal_main_categ,CID10,cid10_main_descricao,procedimento_principal_descricao,codigo_estado,nome_estado,sigla_estado,motivo_saida,motivo_saida_descricao
0,1221200228260,604230028,0.00,120020,{{{}||,19,M,3,21,NaN,...,N,F20,F20,Esquizofrenia,OLANZAPINA 10 MG (POR COMPRIMIDO),12,Acre,AC,21,Permanência por características próprias da d...
1,1221200222231,604230028,0.00,120040,{{~~|,14,M,3,21,NaN,...,N,F20,F20,Esquizofrenia,OLANZAPINA 10 MG (POR COMPRIMIDO),12,Acre,AC,21,Permanência por características próprias da d...
2,1221200245089,604100019,0.00,120040,{}{{~{,48,F,3,21,NaN,...,N,E23,E23,Hipofunção e outros transtornos da hipófise,"DESMOPRESSINA 0,1 MG/ML APLICACAO NASAL (POR F...",12,Acre,AC,21,Permanência por características próprias da d...
3,1221200245232,604110030,256.58,120040,{{~~~||,10,F,3,21,NaN,...,N,E22,E22,Hiperfunção da hipófise,"LEUPRORRELINA 3,75 MG INJETAVEL (POR FRASCO-AM...",12,Acre,AC,21,Permanência por características próprias da d...
4,1221200251854,604110030,256.58,120040,{{~{{|},8,F,3,21,NaN,...,N,E22,E22,Hiperfunção da hipófise,"LEUPRORRELINA 3,75 MG INJETAVEL (POR FRASCO-AM...",12,Acre,AC,21,Permanência por características próprias da d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27162599,1722200329620,604030037,0.00,170210,{~{|~{{,37,M,1,21,NaN,...,N,E22,E22,Hiperfunção da hipófise,"CABERGOLINA 0,5 MG (POR COMPRIMIDO)",17,Tocantins,TO,21,Permanência por características próprias da d...
27162600,1722200326715,604260016,0.00,172100,{{|{|{}{,42,M,1,21,NaN,...,N,N18,N18,Insuficiência renal crônica,SACARATO DE HIDROXIDO FERRICO 100 MG INJETAVEL...,17,Tocantins,TO,21,Permanência por características próprias da d...
27162601,1722200314769,604610017,0.00,172100,{{}|}||,11,M,1,21,NaN,...,N,E23,E23,Hipofunção e outros transtornos da hipófise,SOMATROPINA 4 UI INJETAVEL (POR FRASCO-AMPOLA),17,Tocantins,TO,21,Permanência por características próprias da d...
27162602,1722200313746,604230010,0.00,172100,{{{{},52,F,3,21,NaN,...,N,F20,F20,Esquizofrenia,OLANZAPINA 5 MG (POR COMPRIMIDO),17,Tocantins,TO,21,Permanência por características próprias da d...


### Load

Our load will be just export to `.csv`. Nice :)

In [42]:
codes, uniques = pd.factorize(combined_df['cns_paciente'])
combined_df['cns_coded'] = ['cns_' + f'{code:08}' for code in codes]

combined_df.to_csv('../data/processed/apac-am-2022.csv')

In [43]:
file_path = Path('../data/processed/apac-am-2022.csv')
size_bytes = file_path.stat().st_size
print('../data/processed/apac-am-2022.csv')
print(f"Size in GB: {size_bytes / (1024 ** 3):.2f}")

../data/processed/apac-am-2022.csv
Size in GB: 6.76


In [44]:
toy_data = combined_df.sample(frac = 0.1, axis = 0)
toy_data.to_csv('../data/processed/apac-am-2022-toy.csv', index = False)

file_path = Path('../data/processed/apac-am-2022-toy.csv')
size_bytes = file_path.stat().st_size
print('../data/processed/apac-am-2022-toy.csv')
print(f"Size in GB: {size_bytes / (1024 ** 3):.2f}")

../data/processed/apac-am-2022-toy.csv
Size in GB: 0.65
